In [1]:
import re
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
#from convertTxtToDFFns import parseFullDay
from horseInfoFns import parseHorseInfo
from genInfoFns import parseGenInfo
from horsePlayground import parseFullDay, parseRace
from timesInfoFns import parseTimeInfo

Fractional Times: 21.32 42.82 1:06.20 Final Time: 1:12.69

 Fractional Times: 25.07 49.17 1:14.36 1:27.78 Final Time: 1:41.92

 Fractional Times: 22.05 45.51 58.44 Final Time: 1:12.66

 Fractional Times: 24.15 48.39 1:12.78 1:24.55 Final Time: 1:36.38

 Fractional Times: 22.30 45.62 1:09.67 Final Time: 1:16.13

 Fractional Times: 23.66 47.55 1:12.32 1:24.64 Final Time: 1:37.49

 Fractional Times: 21.35 43.46 1:06.52 Final Time: 1:12.58

 Fractional Times: 22.58 45.76 57.73 Final Time: 1:10.23

 Fractional Times: 21.90 44.67 1:07.58 Final Time: 1:13.70



In [2]:
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.112.txt') as file:
    test1 = file.readlines()
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.148.txt') as file:
    test2 = file.readlines()

In [3]:
jack = parseFullDay(test1)

Fractional Times: 22.09 45.87 Final Time: 52.23

 Final Time: 45.65

 Fractional Times: 22.92 46.57 Final Time: 52.51

 Final Time: 45.23



In [5]:
jack.shape

(21, 50)

In [5]:
line = ' Run-Up: 131 feet Temporary Rail: 30 feet'

In [6]:
def parseRunUp(line):
    fullSearch = re.search(r'Run-Up: ([0-9.]*)', line)
    runUp = fullSearch.group(1)

    return runUp

In [7]:
parseRunUp(line)

'131'